## Description

In [ ]:
"""
All functions for interfacing the course_schedule database in various ways.
Final Project for CSC 336 - Database Systems (SPRING 2022)

@author Gaurav Kuwar, Zibin Luo, Usman Abbas
@version 1.00 May, 2022
"""

### Import Modules

In [3]:
# if the module is not installed yet, we pip install it
# pip install mysql-connector-python
import mysql.connector

import numpy as np
import pandas as pd

### Get config Info

In [379]:
def getConfig(authFn='auth.txt'):
    """
        Reads and formats data from authFn returns it.
    """
    # Read the authentication file
    auth = np.loadtxt('auth.txt', dtype = str)
    # Config dictionary
    labels = ['username', 'password', 'host', 'database']
    config = dict(zip(labels, list(auth)))
    return config

config = getConfig()
config

{'username': 'username',
 'password': 'P@33W0rd',
 'host': '127.0.0.1',
 'database': 'course_schedule'}

### Helper Functions

In [262]:
def colsToQuery(colsNameVal, query_type, separator):
    """
        Takes a dict colsNameVal, and puts into a query format for WHERE, SET parts
        of the query.
    """
    
    query_part = ""
    if colsNameVal: # do nothing if colsNameVal empty
        query_part += " {} ".format(query_type)
        count = len(colsNameVal) # keeps count of how many colsNameVal were processed
        
        for col_name in colsNameVal:
            col_val = colsNameVal[col_name]
            if type(col_val) == str:
                col_val = '"{}"'.format(col_val) # is col_val is a str makes sure its under quotes
            
            query_part += col_name + " = " + str(col_val) # col_name = col_val
            
            if count > 1: # makes sure count values are greater than 1 to and a separator
                query_part += " {} ".format(separator) # Type of separtor, such as AND or , (comma)
            count -= 1
            
    return query_part

In [235]:
def addWhere(colsNameVal, separator="AND"):
    """
        Builds the where part of the query using values in dict colsNameVal
        and uses the separator "AND" by default.
    """
    return colsToQuery(colsNameVal, "WHERE", separator=separator)

In [263]:
# Examples Usage
addWhere({"room_id": 1, "room_num": "ABC"})

' WHERE room_id = 1 AND room_num = "ABC"'

In [236]:
def addSet(colsNameVal):
    """
        Builds the set part of the query using values in dict colsNameVal
        and uses the separator ",".
    """
    return colsToQuery(colsNameVal, "SET", ",")

In [264]:
# Examples Usage
addSet({"room_id": 1, "room_num": "ABC"})

' SET room_id = 1 , room_num = "ABC"'

In [231]:
def getColNameVal(args_dict, exclude=["config"]):
    """
        Gets a dict args_dict and filters values in the args_dict that are None
        or they are in the exclude list.
    """
    result = {}
    for col_name in args_dict:
        if (args_dict[col_name] != None) and (col_name not in exclude):
            result[col_name] = args_dict[col_name]
    return result

## Populate Relations from csv file

In [464]:
def populateRoom(csvfile='room.csv', config=config):
    """
        Populates Room relation with csvfile.
    """
    
    rooms = pd.read_csv(csvfile) # read csv file with pandas
    conn = mysql.connector.connect(**config)
    cursor = conn.cursor() # Cursor

    for i in range(len(rooms)):
        room_num=rooms.iloc[i][0]
        building_name=rooms.iloc[i][1]

        query="INSERT INTO `Room` (room_num, building_name) VALUES ({room_num}, '{building_name}')".\
            format(room_num=room_num,
                   building_name=building_name)

        print("Executing:",query)
        cursor.execute(query)

    cursor.execute('commit')
    cursor.close()
    conn.close()

In [465]:
populateRoom()

Executing: INSERT INTO `Room` (room_num, building_name) VALUES (1, 'NAC')
Executing: INSERT INTO `Room` (room_num, building_name) VALUES (2, 'Marshak')
Executing: INSERT INTO `Room` (room_num, building_name) VALUES (3, 'NAC')
Executing: INSERT INTO `Room` (room_num, building_name) VALUES (4, 'Marshak')
Executing: INSERT INTO `Room` (room_num, building_name) VALUES (5, 'NAC')


In [466]:
def populateCourse(csvfile='course.csv', config=config):
    """
        Populates Course relation with csvfile.
    """
    
    courses = pd.read_csv(csvfile)
    conn = mysql.connector.connect(**config)
        # Cursor
    cursor = conn.cursor()
    
    for i in range(len(courses)):
        course_num=courses.iloc[i][0]
        department=courses.iloc[i][1]
        title=courses.iloc[i][2]
        
        query=("INSERT INTO `Course` (course_num, department, title)"
               " VALUES ({course_num}, '{department}', '{title}');").format(
            course_num=course_num,department=department,title=title)
        print("Executing:",query)
        cursor.execute(query)
        
    cursor.execute('commit')
    cursor.close()
    conn.close()

In [467]:
populateCourse()

Executing: INSERT INTO `Course` (course_num, department, title) VALUES (336, 'CSC', 'Database System');
Executing: INSERT INTO `Course` (course_num, department, title) VALUES (217, 'MATH', 'Probability and Statistics');
Executing: INSERT INTO `Course` (course_num, department, title) VALUES (113, 'CSC', 'Progamming Language');
Executing: INSERT INTO `Course` (course_num, department, title) VALUES (346, 'MATH', 'Linear Algebra');
Executing: INSERT INTO `Course` (course_num, department, title) VALUES (332, 'CSC', 'Operating System');


In [468]:
def populateProfessor(csvfile='professor.csv', config=config):
    """
        Populates Professor relation with csvfile.
    """
    
    professors=pd.read_csv(csvfile)
    conn = mysql.connector.connect(**config) # Cursor
        
    cursor = conn.cursor()
    for i in range(len(professors)):
        first_name=professors.iloc[i][0]
        last_name=professors.iloc[i][1]

        query="INSERT INTO Professor (first_name, last_name) VALUES ('{first_name}','{last_name}')".\
            format(first_name=first_name,last_name=last_name)
        print("Executing:",query)
        cursor.execute(query)
        
    cursor.execute('commit')
    cursor.close()
    conn.close()

In [469]:
populateProfessor()

Executing: INSERT INTO Professor (first_name, last_name) VALUES ('doris','lamb')
Executing: INSERT INTO Professor (first_name, last_name) VALUES ('wade','martinez')
Executing: INSERT INTO Professor (first_name, last_name) VALUES ('bessie','vagas')
Executing: INSERT INTO Professor (first_name, last_name) VALUES ('rose','white')
Executing: INSERT INTO Professor (first_name, last_name) VALUES ('besty','bates')


In [470]:
def populateClass(csvfile='class.csv', config=config):
    """
        Populates Class relation with csvfile.
    """
    classes=pd.read_csv(csvfile)
    conn = mysql.connector.connect(**config) # Cursor
    cursor = conn.cursor()
    for i in range(len(classes)):
        
        section_id=classes.iloc[i][0]
        room_id=classes.iloc[i][1]
        course_id=classes.iloc[i][2]
        year=classes.iloc[i][3]
        term=classes.iloc[i][4]
        professor_id=classes.iloc[i][5]
        start_time=classes.iloc[i][6]
        end_time=classes.iloc[i][7]
        days_of_the_week=classes.iloc[i][8]

        query="INSERT INTO Class VALUES ({section_id},{room_id},{course_id},'{year}','{term}',{professor_id},'{start_time}','{end_time}','{days_of_the_week}')".\
            format(section_id=section_id,room_id=room_id,course_id=course_id,year=year,term=term,professor_id=professor_id,start_time=start_time,
                  end_time=end_time,days_of_the_week=days_of_the_week)
        print("Executing:",query)
        cursor.execute(query)
        
    cursor.execute('commit')
    cursor.close()
    conn.close()

In [471]:
populateClass()

Executing: INSERT INTO Class VALUES (0,1,1,'2022','SPRING',1,'12:00:00','14:00:00','TR')
Executing: INSERT INTO Class VALUES (1,2,1,'2022','SUMMER',3,'14:00:00','15:30:00','TH')
Executing: INSERT INTO Class VALUES (3,2,2,'2019','WINTER',2,'8:00:00','10:00:00','TH')
Executing: INSERT INTO Class VALUES (4,4,4,'2023','FALL',5,'9:00:00','11:00:00','MO')
Executing: INSERT INTO Class VALUES (5,5,4,'2021','FALL',4,'8:30:00','10:30:00','TR')


## Select / Print Relations

In [472]:
def selectRelation(relation, where={}, config=config):
    """
        Takes a relation which the name of a relation in the database in config
        and returns all rows in the relation, also has a where argument
        which adds a where part to the query using addWhere when where is non-empty.
    """
    # Connection
    conn = mysql.connector.connect(**config)
    # Cursor
    cursor = conn.cursor()
    # SELECT query
    query = ("SELECT * FROM `{relation}`".format(relation=relation))
    query += addWhere(where)
    
    # Run the query
    cursor.execute(query) 
    col_names = cursor.column_names # get column names
    # Fetch the results into a list
    records = [record for record in cursor]
    
    # Close the cursor
    cursor.close()
    # Close the connection
    conn.close()
    
    return pd.DataFrame(records, columns=col_names) # return as a pandas Dataframe (looks nicer)

In [473]:
def selectProfessors(professor_id=None, first_name=None, last_name=None, config=config):
    """
        Returns select query of Professor relation, when arguments are passed they
        are used to filter with WHERE in the select query.
    """
    return selectRelation("Professor", getColNameVal(locals()), config=config)

In [474]:
selectProfessors()

,professor_id,first_name,last_name
0,1,doris,lamb
1,2,wade,martinez
2,3,bessie,vagas
3,4,rose,white
4,5,besty,bates


In [477]:
def selectCourses(course_num=None, department=None, title=None, config=config):
    """
        Returns select query of Course relation, when arguments are passed they
        are used to filter with WHERE in the select query.
    """
    return selectRelation("Course", getColNameVal(locals()), config=config)

In [478]:
selectCourses()

,course_id,course_num,department,title
0,3,113,CSC,Progamming Language
1,2,217,MATH,Probability and Statistics
2,5,332,CSC,Operating System
3,1,336,CSC,Database System
4,4,346,MATH,Linear Algebra


In [479]:
def selectRooms(room_num=None, building_name=None, config=config):
    """
        Returns select query of Room relation, when arguments are passed they
        are used to filter with WHERE in the select query.
    """
    return selectRelation("Room", getColNameVal(locals()), config=config)

In [480]:
selectRooms()

,room_id,room_num,building_name
0,1,1,NAC
1,2,2,Marshak
2,3,3,NAC
3,4,4,Marshak
4,5,5,NAC


In [481]:
def selectClasses(section_id=None, 
                  course_id=None, 
                  year=None, 
                  term=None, 
                  professor_id=None, 
                  start_time=None, 
                  end_time=None,
                  days_of_the_week=None, 
                  config=config):
    """
        Returns select query of Class relation, when arguments are passed they
        are used to filter with WHERE in the select query.
    """
    return selectRelation("Class", getWhere(locals()), config=config)

In [482]:
selectClasses()

,section_id,room_id,course_id,year,term,professor_id,start_time,end_time,days_of_the_week
0,0,1,1,2022,SPRING,1,0 days 12:00:00,0 days 14:00:00,TR
1,1,2,1,2022,SUMMER,3,0 days 14:00:00,0 days 15:30:00,TH
2,3,2,2,2019,WINTER,2,0 days 08:00:00,0 days 10:00:00,TH
3,4,4,4,2023,FALL,5,0 days 09:00:00,0 days 11:00:00,MO
4,5,5,4,2021,FALL,4,0 days 08:30:00,0 days 10:30:00,TR


### Specific select functions

In [483]:
def roomsInBuilding(building_name, config=config):
    """
        Get all rooms in a building
    """
    return selectRooms(building_name=building_name, config=config)

In [484]:
roomsInBuilding("NAC")

,room_id,room_num,building_name
0,1,1,NAC
1,3,3,NAC
2,5,5,NAC


In [485]:
def selectDistinct(col_name, relation, config=config):
    """
        Get list of distinct col_name in relation
    """
    # Connection
    conn = mysql.connector.connect(**config)
    # Cursor
    cursor = conn.cursor()
    # SELECT DISTINCT query
    query = ("SELECT DISTINCT {col_name} FROM `{relation}`;".format(relation=relation, col_name=col_name))
    
    # Run the query
    cursor.execute(query) 
    col_names = cursor.column_names # get column names
    # Fetch the results into a list
    records = [record[0] for record in cursor]
    
    # Close the cursor
    cursor.close()
    # Close the connection
    conn.close()
    
    return records

In [486]:
def allBuildings(config=config):
    """
        Get list of all buildings in Room relation 
    """
    return selectDistinct(col_name="building_name", relation="Room")

In [487]:
allBuildings()

['NAC', 'Marshak']

In [488]:
def allDepartments(config=config):
    """
        Get list of all departments in Course relation 
    """
    return selectDistinct(col_name="department", relation="Course")

In [489]:
allDepartments()

['CSC', 'MATH']

In [490]:
def classesByYear(year, config=config):
    return selectClasses(year=year)

In [492]:
classesByYear("2022")

,section_id,room_id,course_id,year,term,professor_id,start_time,end_time,days_of_the_week
0,0,1,1,2022,SPRING,1,0 days 12:00:00,0 days 14:00:00,TR
1,1,2,1,2022,SUMMER,3,0 days 14:00:00,0 days 15:30:00,TH


In [493]:
def classesBySemester(year, term, config=config):
    return selectClasses(year=year, term=term)

In [494]:
classesBySemester("2022", "SPRING")

,section_id,room_id,course_id,year,term,professor_id,start_time,end_time,days_of_the_week
0,0,1,1,2022,SPRING,1,0 days 12:00:00,0 days 14:00:00,TR


In [495]:
def allSectionsOfCourseBySemester(course_num, department, year, term, config=config):
    """
        Returns a the sections for a specific course by semester
        using course_num, department, year, term
    """
    
    where= getColNameVal(locals())
    
    # Connection
    conn = mysql.connector.connect(**config)
    # Cursor
    cursor = conn.cursor()
    # SELECT query
    query = ("SELECT * FROM Class"
            " INNER JOIN Course"
            " ON Class.course_id=Course.course_id")
    
    query += addWhere(where)
    
    # Run the query
    cursor.execute(query) 
    col_names = cursor.column_names # get column names
    # Fetch the results into a list
    records = [record for record in cursor]
    
    # Close the cursor
    cursor.close()
    # Close the connection
    conn.close()
    
    return pd.DataFrame(records, columns=col_names) # return as a pandas Dataframe (looks nicer)

In [496]:
allSectionsOfCourseBySemester(course_num=336, department="CSC", year="2022", term="SPRING")

,section_id,room_id,course_id,year,term,professor_id,start_time,end_time,days_of_the_week,course_id,course_num,department,title
0,0,1,1,2022,SPRING,1,0 days 12:00:00,0 days 14:00:00,TR,1,336,CSC,Database System


In [546]:
def classInfo(section_id, department, course_num, year, term):
    """
        Returns all information a Student would need for attending a class, section_id, 
        department, course_num, title, term, year, days_of_the_week, start_time, 
        end_time, room_num, building_name, professor name, using join on multiple relations
    """
    
    where= getColNameVal(locals())
    
    # Connection
    conn = mysql.connector.connect(**config)
    # Cursor
    cursor = conn.cursor()
    # SELECT query
    query = ("SELECT section_id, department, course_num, title,"
            " term, year, days_of_the_week, start_time, end_time, room_num, building_name,"
            " first_name, last_name"
            " FROM Class"
            " INNER JOIN Course"
            " ON Class.course_id=Course.course_id"
            " INNER JOIN Room"
            " ON Class.room_id=Room.room_id"
            " INNER JOIN Professor"
            " ON Class.professor_id=Professor.professor_id")
    
    query += addWhere(where)
    
    # Run the query
    cursor.execute(query) 
    col_names = cursor.column_names # get column names
    # Fetch the results into a list
    records = [record for record in cursor]
    
    # Close the cursor
    cursor.close()
    # Close the connection
    conn.close()
    
    return pd.DataFrame(records, columns=col_names) # return as a pandas Dataframe (looks nicer)

In [548]:
classInfo(section_id=5, department="SCI", course_num=346, year="2021", term="FALL")

,section_id,department,course_num,title,term,year,days_of_the_week,start_time,end_time,room_num,building_name,first_name,last_name
0,5,SCI,346,Linear Algebra,FALL,2021,TR,0 days 08:30:00,0 days 10:30:00,5,NAC,rose,white


## Insert Records into Relation

In [499]:
def addRoom(room_num, building_name, config=config):
    """
        Insert new record to Room relation
    """
    # Connection
    conn = mysql.connector.connect(**config)
    # Cursor
    cursor = conn.cursor()
    relation = "Room"

    # SELECT query

    query = ("INSERT INTO `{relation}` (room_num, building_name) VALUES ({room_num}, \"{building_name}\");".format(
        relation=relation,
        room_num=room_num,
        building_name=building_name))

    # Run the query
    cursor.execute(query)

    # Committing the changes    
    cursor.execute('commit')
    
    print("Successfully Executed:", query)

    # Close the cursor
    cursor.close()
    # Close the connection
    conn.close()

In [500]:
addRoom(room_num="10", building_name="Grove")

Successfully Executed: INSERT INTO `Room` (room_num, building_name) VALUES (10, "Grove");


In [501]:
selectRooms()

,room_id,room_num,building_name
0,1,1,NAC
1,2,2,Marshak
2,3,3,NAC
3,4,4,Marshak
4,5,5,NAC
5,6,10,Grove


In [502]:
selectProfessors()

,professor_id,first_name,last_name
0,1,doris,lamb
1,2,wade,martinez
2,3,bessie,vagas
3,4,rose,white
4,5,besty,bates


In [503]:
def addProfessor(first_name, last_name, config=config):
    """
        Insert new record to Professor relation
    """
     # Connection
    conn = mysql.connector.connect(**config)
    # Cursor
    cursor = conn.cursor()
    relation = "Professor"

    # SELECT query

    query = ("INSERT INTO `{relation}` (first_name, last_name) VALUES (\"{first_name}\", \"{last_name}\");".format(
        relation=relation,
        first_name=first_name,
        last_name=last_name))

    # Run the query
    cursor.execute(query)

    # Committing the changes    
    cursor.execute('commit')
    
    print("Successfully Executed:", query)

    # Close the cursor
    cursor.close()
    # Close the connection
    conn.close()

In [504]:
addProfessor("Rice", "Water")

Successfully Executed: INSERT INTO `Professor` (first_name, last_name) VALUES ("Rice", "Water");


In [505]:
selectProfessors()

,professor_id,first_name,last_name
0,1,doris,lamb
1,2,wade,martinez
2,3,bessie,vagas
3,4,rose,white
4,5,besty,bates
5,6,Rice,Water


In [506]:
selectCourses()

,course_id,course_num,department,title
0,3,113,CSC,Progamming Language
1,2,217,MATH,Probability and Statistics
2,5,332,CSC,Operating System
3,1,336,CSC,Database System
4,4,346,MATH,Linear Algebra


In [507]:
def addCourse(course_num, department, title, config=config):
    """
        Insert new record to Course relation
    """
     # Connection
    conn = mysql.connector.connect(**config)
    # Cursor
    cursor = conn.cursor()
    relation = "Course"

    # SELECT query

    query = ("INSERT INTO `{relation}` (course_num, department,title) VALUES (\"{course_num}\", \"{department}\",\"{title}\");".format(
        relation=relation,
        course_num=course_num,
        department=department,
        title=title))

    # Run the query
    cursor.execute(query)

    # Committing the changes    
    cursor.execute('commit')
    
    print("Successfully Executed:", query)

    # Close the cursor
    cursor.close()
    # Close the connection
    conn.close()

In [508]:
addCourse("322", "CSC", "Software Engineering")

Successfully Executed: INSERT INTO `Course` (course_num, department,title) VALUES ("322", "CSC","Software Engineering");


In [509]:
selectCourses()

,course_id,course_num,department,title
0,3,113,CSC,Progamming Language
1,2,217,MATH,Probability and Statistics
2,6,322,CSC,Software Engineering
3,5,332,CSC,Operating System
4,1,336,CSC,Database System
5,4,346,MATH,Linear Algebra


In [510]:
selectClasses()

,section_id,room_id,course_id,year,term,professor_id,start_time,end_time,days_of_the_week
0,0,1,1,2022,SPRING,1,0 days 12:00:00,0 days 14:00:00,TR
1,1,2,1,2022,SUMMER,3,0 days 14:00:00,0 days 15:30:00,TH
2,3,2,2,2019,WINTER,2,0 days 08:00:00,0 days 10:00:00,TH
3,4,4,4,2023,FALL,5,0 days 09:00:00,0 days 11:00:00,MO
4,5,5,4,2021,FALL,4,0 days 08:30:00,0 days 10:30:00,TR


In [511]:
def addClass(section_id,
             room_id,
             course_id,
             year,
             term,
             professor_id,
             start_time,
             end_time,
             days_of_the_week, 
             config=config):
    """
        Insert new record to Class relation
    """
    # Connection
    conn = mysql.connector.connect(**config)
    # Cursor
    cursor = conn.cursor()
    relation = "Class"

    # SELECT query

    query = ("INSERT INTO `{relation}`" 
             " (section_id, room_id, course_id, year, term,"
             " professor_id, start_time,end_time,days_of_the_week)"
             " VALUES ({section_id}, {room_id}, {course_id}, \"{year}\","
             " \"{term}\", {professor_id}, \"{start_time}\","
             " \"{end_time}\",\"{days_of_the_week}\");".format(
            relation=relation,
            section_id=section_id,
            room_id=room_id,
            course_id=course_id,
            year=year,
            term=term,
            professor_id=professor_id,
            start_time=start_time,
            end_time=end_time,
            days_of_the_week=days_of_the_week))
    
    print(query)
    # Run the query
    cursor.execute(query)

    # Committing the changes    
    cursor.execute('commit')
    
    print("Successfully Executed:", query)


    # Close the cursor
    cursor.close()
    # Close the connection
    conn.close()

In [512]:
addClass(section_id=3,
         room_id=1,
         course_id=6,
         year="2018",
         term="WINTER",
         professor_id=1,
         start_time="12:00:00",
         end_time="14:00:00",
         days_of_the_week="MTW")

INSERT INTO `Class` (section_id, room_id, course_id, year, term, professor_id, start_time,end_time,days_of_the_week) VALUES (3, 1, 6, "2018", "WINTER", 1, "12:00:00", "14:00:00","MTW");
Successfully Executed: INSERT INTO `Class` (section_id, room_id, course_id, year, term, professor_id, start_time,end_time,days_of_the_week) VALUES (3, 1, 6, "2018", "WINTER", 1, "12:00:00", "14:00:00","MTW");


In [513]:
selectClasses()

,section_id,room_id,course_id,year,term,professor_id,start_time,end_time,days_of_the_week
0,0,1,1,2022,SPRING,1,0 days 12:00:00,0 days 14:00:00,TR
1,1,2,1,2022,SUMMER,3,0 days 14:00:00,0 days 15:30:00,TH
2,3,2,2,2019,WINTER,2,0 days 08:00:00,0 days 10:00:00,TH
3,3,1,6,2018,WINTER,1,0 days 12:00:00,0 days 14:00:00,MTW
4,4,4,4,2023,FALL,5,0 days 09:00:00,0 days 11:00:00,MO
5,5,5,4,2021,FALL,4,0 days 08:30:00,0 days 10:30:00,TR


## Delete Records from Relations

In [514]:
selectRooms()

,room_id,room_num,building_name
0,1,1,NAC
1,2,2,Marshak
2,3,3,NAC
3,4,4,Marshak
4,5,5,NAC
5,6,10,Grove


In [515]:
def deleteRoom(room_id, config=config):
    """
        Delete a row from class relation
    """
    # Connection
    conn = mysql.connector.connect(**config)
    # Cursor
    cursor = conn.cursor()
    #Name of the relation
    relation = "Room"

    # SELECT query

    query = ("DELETE FROM `{relation}` WHERE room_id = {room_id};".format(
        #Primary keys needed for deletion
        relation=relation,
        room_id=room_id))

    # Run the query
    cursor.execute(query)

    # Committing the changes    
    cursor.execute('commit')
    
    print("Successfully Executed:", query)


    # Close the cursor
    cursor.close()
    # Close the connection
    conn.close()

In [517]:
deleteRoom(room_id=6) # deleted a Room record that is not used in Class relation

Successfully Executed: DELETE FROM `Room` WHERE room_id = 6;


In [518]:
selectRooms()

,room_id,room_num,building_name
0,1,1,NAC
1,2,2,Marshak
2,3,3,NAC
3,4,4,Marshak
4,5,5,NAC


In [519]:
selectProfessors()

,professor_id,first_name,last_name
0,1,doris,lamb
1,2,wade,martinez
2,3,bessie,vagas
3,4,rose,white
4,5,besty,bates
5,6,Rice,Water


In [520]:
def deleteProfessor(professor_id, config=config):
    """
        Delete a row from class relation
    """
    # Connection
    conn = mysql.connector.connect(**config)
    # Cursor
    cursor = conn.cursor()
    #Name of the relation
    relation = "Professor"

    # SELECT query

    query = ("DELETE FROM `{relation}` WHERE professor_id = {professor_id};".format(
        #Primary keys needed for deletion
        relation=relation,
        professor_id=professor_id))

    # Run the query
    cursor.execute(query)

    # Committing the changes    
    cursor.execute('commit')
    
    print("Successfully Executed:", query)


    # Close the cursor
    cursor.close()
    # Close the connection
    conn.close()

In [521]:
deleteProfessor(professor_id=6) # deleted a Professor record that is not used in Class relation

Successfully Executed: DELETE FROM `Professor` WHERE professor_id = 6;


In [522]:
selectProfessors()

,professor_id,first_name,last_name
0,1,doris,lamb
1,2,wade,martinez
2,3,bessie,vagas
3,4,rose,white
4,5,besty,bates


In [523]:
selectCourses()

,course_id,course_num,department,title
0,3,113,CSC,Progamming Language
1,2,217,MATH,Probability and Statistics
2,6,322,CSC,Software Engineering
3,5,332,CSC,Operating System
4,1,336,CSC,Database System
5,4,346,MATH,Linear Algebra


In [340]:
def deleteCourse(course_id, config=config):
    """
        Delete a row from class relation
    """
    # Connection
    conn = mysql.connector.connect(**config)
    # Cursor
    cursor = conn.cursor()
    #Name of the relation
    relation = "Course"

    # SELECT query

    query = ("DELETE FROM `{relation}` WHERE course_id = {course_id};".format(
        #Primary keys needed for deletion
        relation=relation,
        course_id=course_id)) 

    # Run the query
    cursor.execute(query)

    # Committing the changes    
    cursor.execute('commit')
    
    print("Successfully Executed:", query)


    # Close the cursor
    cursor.close()
    # Close the connection
    conn.close()

In [524]:
deleteCourse(course_id=5) # deleted a Course record that is not used in Class relation

In [525]:
selectCourses()

,course_id,course_num,department,title
0,3,113,CSC,Progamming Language
1,2,217,MATH,Probability and Statistics
2,6,322,CSC,Software Engineering
3,1,336,CSC,Database System
4,4,346,MATH,Linear Algebra


In [526]:
selectClasses()

,section_id,room_id,course_id,year,term,professor_id,start_time,end_time,days_of_the_week
0,0,1,1,2022,SPRING,1,0 days 12:00:00,0 days 14:00:00,TR
1,1,2,1,2022,SUMMER,3,0 days 14:00:00,0 days 15:30:00,TH
2,3,2,2,2019,WINTER,2,0 days 08:00:00,0 days 10:00:00,TH
3,3,1,6,2018,WINTER,1,0 days 12:00:00,0 days 14:00:00,MTW
4,4,4,4,2023,FALL,5,0 days 09:00:00,0 days 11:00:00,MO
5,5,5,4,2021,FALL,4,0 days 08:30:00,0 days 10:30:00,TR


In [527]:
def deleteClass(section_id, course_id, room_id, year, term, config=config): 
    """
        Delete a row from class relation
    """
    # Connection
    conn = mysql.connector.connect(**config)
    # Cursor connection
    cursor = conn.cursor()
    #Name of the relation
    relation = "Class" 

    # SELECT query
    query = ("DELETE FROM {relation} WHERE (section_id, course_id, room_id, year, term) = ({section_id}, " \
       "{course_id}, {room_id}, '{year}', '{term}');".format(
        #Primary keys needed for deletion
        relation=relation,
        section_id=section_id,
        course_id=course_id,
        room_id=room_id,
        year=year,
        term=term)) 

    # Run the query
    cursor.execute(query)

    # Committing the changes    
    cursor.execute('commit')
    
    print("Successfully Executed:", query)


    # Close the cursor
    cursor.close()
    # Close the connection
    conn.close()

In [529]:
deleteClass(section_id=0, course_id=1, room_id=1, year="2022", term="SPRING")

Successfully Executed: DELETE FROM Class WHERE (section_id, course_id, room_id, year, term) = (0, 1, 1, '2022', 'SPRING');


In [530]:
selectClasses()

,section_id,room_id,course_id,year,term,professor_id,start_time,end_time,days_of_the_week
0,1,2,1,2022,SUMMER,3,0 days 14:00:00,0 days 15:30:00,TH
1,3,2,2,2019,WINTER,2,0 days 08:00:00,0 days 10:00:00,TH
2,3,1,6,2018,WINTER,1,0 days 12:00:00,0 days 14:00:00,MTW
3,4,4,4,2023,FALL,5,0 days 09:00:00,0 days 11:00:00,MO
4,5,5,4,2021,FALL,4,0 days 08:30:00,0 days 10:30:00,TR


## Update Relations

In [531]:
def updateRelation(unique_cols, relation, col_names, config=config):
    """
        Update a row from class relation
    """
    
    set_part = addSet(col_names)
    where_part = addWhere(unique_cols)
    
    # Connection
    conn = mysql.connector.connect(**config)
    # Cursor connection
    cursor = conn.cursor()
    
    # SELECT query
    query = "UPDATE `{relation}` {set_part} {where_part};".format(
        relation=relation,
        where_part=where_part,
        set_part=set_part)
    
    # Run the query
    cursor.execute(query)

    # Committing the changes
    cursor.execute('commit')

    # Close the cursor
    cursor.close()
    # Close the connection
    conn.close()
    
    return "Update Successful!"

In [532]:
selectRooms()

,room_id,room_num,building_name
0,1,1,NAC
1,2,2,Marshak
2,3,3,NAC
3,4,4,Marshak
4,5,5,NAC


In [362]:
def updateRoom(room_id, 
               room_num=None, 
               building_name=None, 
               config=config):
    
    col_names = getColNameVal(locals(), exclude=["config", "room_id"])
    relation = "Room"
    unique_cols = {"room_id": room_id}
    
    return updateRelation(unique_cols, relation, col_names, config=config)

In [533]:
updateRoom(room_id=1, building_name="Grove")

'Update Successful!'

In [534]:
selectRooms()

,room_id,room_num,building_name
0,1,1,Grove
1,2,2,Marshak
2,3,3,NAC
3,4,4,Marshak
4,5,5,NAC


In [535]:
selectProfessors()

,professor_id,first_name,last_name
0,1,doris,lamb
1,2,wade,martinez
2,3,bessie,vagas
3,4,rose,white
4,5,besty,bates


In [366]:
def updateProfessor(professor_id, 
                    first_name=None, 
                    last_name=None, 
                    config=config):
    
    col_names = getColNameVal(locals(), exclude=["config", "professor_id"])
    relation = "Professor"
    unique_cols = {"professor_id": professor_id}
    
    return updateRelation(unique_cols, relation, col_names, config=config)

In [536]:
updateProfessor(professor_id=5, first_name="Phone")

'Update Successful!'

In [537]:
selectProfessors()

,professor_id,first_name,last_name
0,1,doris,lamb
1,2,wade,martinez
2,3,bessie,vagas
3,4,rose,white
4,5,Phone,bates


In [538]:
selectCourses()

,course_id,course_num,department,title
0,3,113,CSC,Progamming Language
1,2,217,MATH,Probability and Statistics
2,6,322,CSC,Software Engineering
3,1,336,CSC,Database System
4,4,346,MATH,Linear Algebra


In [539]:
def updateCourse(course_id, 
                 course_num=None, 
                 department=None, 
                 title=None,
                 config=config):
    
    col_names = getColNameVal(locals(), exclude=["config", "course_id"])
    relation = "Course"
    unique_cols = {"course_id": course_id}
    
    return updateRelation(unique_cols, relation, col_names, config=config)

In [540]:
updateCourse(course_id=4, department="SCI")

'Update Successful!'

In [541]:
selectCourses()

,course_id,course_num,department,title
0,3,113,CSC,Progamming Language
1,2,217,MATH,Probability and Statistics
2,6,322,CSC,Software Engineering
3,1,336,CSC,Database System
4,4,346,SCI,Linear Algebra


In [542]:
selectClasses()

,section_id,room_id,course_id,year,term,professor_id,start_time,end_time,days_of_the_week
0,1,2,1,2022,SUMMER,3,0 days 14:00:00,0 days 15:30:00,TH
1,3,2,2,2019,WINTER,2,0 days 08:00:00,0 days 10:00:00,TH
2,3,1,6,2018,WINTER,1,0 days 12:00:00,0 days 14:00:00,MTW
3,4,4,4,2023,FALL,5,0 days 09:00:00,0 days 11:00:00,MO
4,5,5,4,2021,FALL,4,0 days 08:30:00,0 days 10:30:00,TR


In [543]:
def updateClass(section_id, course_id, year, term,
                 room_id=None,
                 professor_id=None,
                 start_time=None, 
                 end_time=None,
                 days_of_the_week=None,
                 config=config):
    
    col_names = getColNameVal(locals(), exclude=["config", "course_id", "section_id", "year", "term"])
    relation = "Class"
    unique_cols = {"section_id": section_id, "course_id": course_id, "year": year, "term": term}
    
    return updateRelation(unique_cols, relation, col_names, config=config)

In [544]:
updateClass(section_id=2, course_id=1, year="2022", term="SPRING", days_of_the_week="RF")

'Update Successful!'

In [545]:
selectClasses()

,section_id,room_id,course_id,year,term,professor_id,start_time,end_time,days_of_the_week
0,1,2,1,2022,SUMMER,3,0 days 14:00:00,0 days 15:30:00,TH
1,3,2,2,2019,WINTER,2,0 days 08:00:00,0 days 10:00:00,TH
2,3,1,6,2018,WINTER,1,0 days 12:00:00,0 days 14:00:00,MTW
3,4,4,4,2023,FALL,5,0 days 09:00:00,0 days 11:00:00,MO
4,5,5,4,2021,FALL,4,0 days 08:30:00,0 days 10:30:00,TR
